# Connect PostgreSQL Server

In [61]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os
import glob
import csv
import configparser

In [62]:
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
# Build connection string
server_name = os.getenv('SERVER_NAME')
user_name = os.getenv('USER_NAME')
password = os.getenv('PASSWORD')
server_port = os.getenv('PORT')

#postgres is the master database you will connect to before you have created and switched to any other databases
master_db = 'postgres'
connection_string = f'host={server_name} port={server_port} dbname={master_db} user={user_name} password={password} sslmode=require'


In [46]:
# Open connection
pg_session = psycopg2.connect(connection_string)
#Auto commit -> No transaction is started when command are issued and no commit() or rollback() is required.
pg_session.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
print("Open connection to postgresql server successfully")


Open connection to postgresql server successfully


# Create Database (DB)

In [47]:
operational_db = os.getenv('DATABASE')

In [48]:
# Init a SQL processor
sql_processor = pg_session.cursor()
print("Init sql proccessor successfully")

Init sql proccessor successfully


In [49]:
sql_processor.execute(f'DROP DATABASE IF EXISTS {operational_db}')

In [50]:
sql_processor.execute(f"CREATE DATABASE {operational_db}")

In [51]:
sql_processor.close()
pg_session.close()

# Switch 2 Operational DB & Dumping Data

## Switch DB

In [52]:
connection_string = f'host={server_name} port={server_port} dbname={operational_db} user={user_name} password={password} sslmode=require'
print("Open connection to postgresql server successfully")

Open connection to postgresql server successfully


## Dumping

In [53]:
# Get list of paths to data files
filepath = os.getcwd() + '\data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

In [63]:
# Prepare sql
get_drop_exist_table_query = lambda table_name : f"DROP TABLE IF EXISTS {table_name}"
get_create_query = lambda table_name, column_def : f"CREATE TABLE {table_name} ({column_def})"

In [66]:
config = configparser.ConfigParser()
config.read('./csv2postgre.table.ini')
TABLE_NAME_KEY = "table_name"
COLUMN_DEF_KEY = "column_def"


In [68]:
pg_session = psycopg2.connect(connection_string)
try:
    with pg_session.cursor() as sql_processor:
        try:
            for path in config.sections():
            # reading csv file 
                with open(path, 'r', encoding = 'utf8', newline='') as csvfile:
                    table_name = config[path][TABLE_NAME_KEY]
                    column_def = config[path][COLUMN_DEF_KEY]
                    drop_exist_table_query = get_drop_exist_table_query(table_name)
                    create_query = get_create_query(table_name, column_def)
                    sql_processor.execute(drop_exist_table_query)
                    sql_processor.execute(create_query)
                    sql_processor.copy_from(csvfile, table_name, sep=",", null = "")
                    pg_session.commit()
                    print(drop_exist_table_query)
                    print(create_query)
        except (Exception, psycopg2.DatabaseError) as error:
            print('Error: %s' % error)
            pg_session.rollback()
        else:
            print('Finished populating')
finally:
    pg_session.close()

DROP TABLE IF EXISTS payment
CREATE TABLE payment (payment_id INTEGER PRIMARY KEY, date DATE, amount MONEY, rider_id INTEGER)
DROP TABLE IF EXISTS rider
CREATE TABLE rider (rider_id INTEGER PRIMARY KEY, first VARCHAR(50), last VARCHAR(50), address VARCHAR(100), birthday DATE, account_start_date DATE, account_end_date DATE, is_member BOOLEAN)
DROP TABLE IF EXISTS station
CREATE TABLE station (station_id VARCHAR(50) PRIMARY KEY, name VARCHAR(75), latitude FLOAT, longitude FLOAT)
DROP TABLE IF EXISTS trip
CREATE TABLE trip (trip_id VARCHAR(50) PRIMARY KEY, rideable_type VARCHAR(75), start_at TIMESTAMP, ended_at TIMESTAMP, start_station_id VARCHAR(50), end_station_id VARCHAR(50), rider_id INTEGER)
Finished populating
